# Data Smart Sans Excel

< need to add linker to master >

In [1]:
# Download and extract Data Smart files
import os
import urllib.request
import zipfile
if not os.path.exists(os.path.join(os.getcwd(), "data_smart_files")):
    r = range(1, 11)
    url_list = []
    url_base = "http://media.wiley.com/product_ancillary/6X/11186614/DOWNLOAD/"
    for i in r:
        url_list.append(url_base + 'ch' + str(i).zfill(2) + '.zip')
    for url in url_list:
        file_name = url[-8:]
        urllib.request.urlretrieve (url, file_name)
        z = zipfile.ZipFile(file_name, "r")
        z.extractall(os.path.join(os.getcwd(), "data_smart_files"))
        z.close()

## Chapter 3 - Naïve Bayes